In [1]:
import copy
import math
import numpy

from heat_load_calc.external import factor_nu as nu

Importing Jupyter notebook from nu.ipynb


# 基準値計算用の住宅の幾何形状の決定

## 3. 面積の設定

### 1) 用途別床面積を取得する関数

#### Outline

延床面積、主たる居室の床面積、その他の居室の床面積より、非居室の床面積を算出する。

#### Function

In [2]:
def get_area_floor(d_common):
    return {
        'main_occupant_room'  : d_common['main_occupant_room_floor_area'],
        'other_occupant_room' : d_common['other_occupant_room_floor_area'],
        'non_occupant_room'   : d_common['total_floor_area'] \
                                - d_common['main_occupant_room_floor_area'] - d_common['other_occupant_room_floor_area']
    }

### 2) 総外皮面積に対する開口部比率、方位別の開口部比率

#### Outline

総外皮面積に対する開口部比率

| 建て方 | 総外皮面積に対する開口部比率 |
|---|---|
| 戸建 | 11％ |
| 共同 | 7％ |

方位別の開口部比率

| 建て方 | 北 | 南 | 東 | 西 |
|---|---|---|---|---|
| 戸建 | 11.0％ | 68.6％ | 13.2％ | 7.2％ |
| 共同 | 21.5％ | 63.3％ | 0.0％ | 15.2％ |

#### Function

In [3]:
# 開口部比率, -
def get_ratio_window(house_type) :
    
    ratio_window ={
        'detached': {
            'total': 0.11, 
            'direction':{ 'top':0.000, 'n':0.110, 's':0.686, 'e':0.132, 'w':0.072 }
        }, 
        'condominium': { 
            'total': 0.07, 
            'direction':{ 'top':0.000, 'n':0.215, 's':0.633, 'e':0.000, 'w':0.152 }
        }
    }
    
    return ratio_window[house_type]

### 3) 方位別の開口部面積の設定

#### Outline

総外皮面積、相外皮面積に対する開口部比率、方位別の開口部比率より、方位別の開口部面積を設定する。  
ただし、ここで設定するのは暫定値であり、建て方別に、各方位の外皮面積を上限とした調整を行う。

$$
\displaystyle
A_w= A_{total} \times r_w
$$

$$
\displaystyle
A_{w,j}= A_w \times r_{w,j}
$$

$r_{w}$: 総外皮面積に対する開口部比率, -  
$r_{w,j}$: 方位jの開口部比率, -  
$A_{w}$: 開口部面積, m<sup>2</sup>  
$A_{w,j}$: 方位jの開口部面積, m<sup>2</sup>  
$A_{total}$: 総外皮面積, m<sup>2</sup>  

#### Function

In [4]:
def get_area_windows(house_type, area_total_skin) :

    # 開口部比率, -
    ratio_window = get_ratio_window(house_type)
    
    # 開口部面積, m
    area_window = ratio_window['total'] * area_total_skin
    area_windows = { x : area_window * ratio_window['direction'][x] for x in ratio_window['direction'].keys()}
    
    return area_windows

### 4) 方位別の開口部面積の調整

#### Outline

方位別の開口部面積について、各方位の外皮面積を上限とした調整を行う。  
開口部面積の大きい方位より順に、各方位の外皮面積を上回る開口部面積を割り付ける。

In [5]:
# 窓面積の調整
def adjust_window_area(area_windows_pre, priority):

    area_adjust = 0
    area_windows = {}
    for i in range(0,5):
        area_windows[priority[i]['direction']] = min( area_adjust + area_windows_pre[priority[i]['direction']], priority[i]['area_max'] )
        area_adjust = max( 0, area_adjust + area_windows_pre[priority[i]['direction']] - priority[i]['area_max'] )

    return area_windows

In [6]:
area_windows = {'s':10,'n':5,'e':18,'w':6,'top':0}

area_windows = adjust_window_area( area_windows, 
                                  [{'direction':  's','area_max': 8},
                                   {'direction':  'e','area_max': 8},
                                   {'direction':  'n','area_max': 7},
                                   {'direction':  'w','area_max': 7},
                                   {'direction':'top','area_max':10}] )
    
print( area_windows )

{'s': 8, 'e': 8, 'n': 7, 'w': 7, 'top': 9}


### 5) 外壁・開口部・内壁のリスト作成

#### Outline

各部位の面積をもとに、外壁・開口部・内壁のリストを作成する。

#### Function1

外壁のリスト作成

In [7]:
def get_list_general_parts(area_general_parts) :

    correspondence = {
        'type':{ 'ceiling':'ceiling', 'floor':'floor', 'wall_n':'wall', 'wall_s':'wall', 'wall_e':'Wall', 'wall_w':'wall' },
        'direction':{ 'ceiling':'top', 'floor':'bottom', 'wall_n':'n', 'wall_s':'s', 'wall_e':'e', 'wall_w':'w' }
    }
    
    d = []    
    for x in area_general_parts.keys() :
        for y in area_general_parts[x].keys() :
            if area_general_parts[x][y] > 0 :
                d.append({
                    'name'                  : x + '_' + y,
                    'general_part_type'     : correspondence['type'][x],
                    'next_space'            : 'outdoor',
                    'external_surface_type' : 'outdoor',
                    'direction'             : correspondence['direction'][x],
                    'area'                  : area_general_parts[x][y],
                    'space_type'            : y
                })
    
    return d

#### Function2

開口部のリスト作成

In [8]:
def get_list_windows(area_windows) :
    
    d = []    
    for x in area_windows.keys() :
        for y in area_windows[x].keys() :
            if area_windows[x][y] > 0 :
                d.append({
                    'name'       : x + '_' + y,
                    'next_space' : 'outdoor',
                    'direction'  : x,
                    'area'       : area_windows[x][y],
                    'space_type' : y
                })
    return d

#### Function3

内壁のリスト作成

In [9]:
def get_list_inner_walls(area_inner_walls) :
    
    d = []    
    for x in area_inner_walls.keys() :
        if area_inner_walls[x] > 0 :
            d.append({
                'name'      : 'inner_wall_' + x ,
                'area'      : area_inner_walls[x],
                'space_type': x,
                'type'      : 'inner_wall'})

    return d

#### Function4

外壁・開口部・内壁のリストの統合

In [10]:
def get_list(area) :
    
    d = {}
    d['general_parts'] = get_list_general_parts( area['general_parts'] )
    d['windows'] = get_list_windows( area['windows'] )
    d['inner_walls'] = get_list_inner_walls( area['inner_walls'] )
    
    return d

### 6) 基準値計算用の住宅の幾何形状の決定

#### Outline

戸建住宅、共同住宅別に階高、階数を設定し、総外皮面積と床面積が設計住戸と等しくなるような直方体の住宅の幾何形状を決定する。  
ただし、以下の場合には、調整を行う。  
　設計住戸の総外皮面積が比較的小さく、総外皮面積と床面積が設計住戸に等しいという条件を満たす幾何形状を決定できない場合  
　基準値計算用の住戸の平面のアスペクト比が上限値を超える場合

建物高を各階の階高の合計値とする。

$$
\displaystyle
h = \sum_{n}{h_n}
$$

$h$: 建物高, m  
$h_n$: $n$階の階高, m  
$n$: 階数, -  

基準値計算用の住宅の総外皮面積を、設計住戸の総外皮面積、もしくは、正方形プランを仮定した総外皮面積の大きい方とする。

![image.png](attachment:image.png)

$$
\displaystyle
A_{total skin}= \max \left( A_{total skin,design}, \frac{ 2 \times A_{floor}}{ n } +  4 \times h \times　\sqrt{ \frac{ A_{floor} }{ n }} \right)
$$

$A_{floor}$: 延床皮面積, m<sup>2</sup>  
$A_{total skin}$: 基準値計算用の住宅の総外皮面積, m<sup>2</sup>  
$A_{total skin,design}$: 設計住戸の総外皮面積, m<sup>2</sup>  
$h$: 建物高, m  
$n$: 階数, -  

周長を、総外皮面積、延床面積、建物高、階数より算出する。  
短辺・長辺の長さを、周長、延床面積、階数、アスペクト比の上限値より算出する。

$$
\displaystyle
l_{circumference}= \frac{A_{totalskin}- 2 \times \frac{ A_{floor}}{ n }}{h}
$$
$$
\displaystyle
l_{narrow}= \max \left( \frac {l_{circumference}}{ 2 \times (1+r_{aspect,max})}, \frac{l_{circumference} - \sqrt{l_{circumference}^2 - 16 \times \frac{ A_{floor} }{ n }}}{4} \right)
$$
$$
\displaystyle
l_{long}= \frac{l_{circumference} - 2 \times l_{narrow}}{2}
$$

$l_{circumference}$: 周長, m  
$l_{narrow}$: 短辺長さ, m  
$l_{long}$: 長辺長さ, m  
$A_{floor}$: 延床皮面積, m<sup>2</sup>  
$A_{total skin}$: 基準値計算用の住宅の総外皮面積, m<sup>2</sup>  
$r_{aspect,max}$: アスペクト比の上限値（2とする）, -  
$n$: 階数, -  

アスペクト比を上限値により調整している場合に、面積算出用の床面積の調整を行うため、調整係数を算出する。

$$
\displaystyle
r_{area,floor}= \frac{ n \times l_{narrow} \times l_{long} }{ A_{floor} }
$$

$l_{narrow}$: 短辺長さ, m  
$l_{long}$: 長辺長さ, m  
$A_{floor}$: 延床皮面積, m<sup>2</sup>  
$n$: 階数, -  
$r_{area,floor}$: アスペクト比を上限値により調整している場合に、面積算出用の床面積の調整を行うための調整係数, -  

#### Function

In [11]:
def get_geometry(d, area_floor) :
    
    # 階高の設定
    floor_height ={ 'detached' : [ 2.9, 2.7 ], 'condominium' : [ 2.8 ] }[d['common']['house_type']]
    
    # 階数の設定
    n = { 'detached': 2, 'condominium': 1 }[d['common']['house_type']]
        
    # 建物高の算出
    height=sum( x for x in floor_height )
    
    # 基準住戸の総外皮面積の算出
    area_total_skin_min=2*(d['common']['total_floor_area']/n)+4*height*(d['common']['total_floor_area']/n)**0.5
    area_total_skin=max( area_total_skin_min, d['common']['total_outer_skin_area'] )
    
    # 周長の算出
    length_circumference=(area_total_skin-(2*d['common']['total_floor_area']/n))/height
    
    # アスペクト比の上限値の設定    
    ratio_aspect_max=2
    
    # 短辺・長辺長さの算出
    length_narrow=max(length_circumference/((1+ratio_aspect_max)*2),
                      (length_circumference-max( 0, length_circumference**2-16*d['common']['total_floor_area']/n )**0.5)/4)
    length_long=(length_circumference-length_narrow*2)/2
    
    # アスペクト比を上限値により調整している場合、面積算出用の床面積の調整
    area_floor_r = { x : (area_floor[x] * length_narrow * length_long * n / d['common']['total_floor_area']) for x in area_floor.keys()}

    return n, height, floor_height, area_total_skin, length_narrow, length_long, area_floor_r

### 7) 戸建住宅の面積の算出

#### Outline

戸建住宅の各部位の面積を、以下の場合に応じて、算出する。  
　主たる居室とその他の居室の床面積が等しい場合：北側を非居室、南側の1階を主たる居室、2階をその他の居室とする  
　主たる居室の床面積がその他の居室の床面積より大きい場合：北側を非居室、2階の南西をその他の居室とする  
　主たる居室の床面積がその他の居室の床面積より小さい場合：北側を非居室、1階の南東を主たる居室とする  

#### Function1

主たる居室とその他の居室の床面積が等しい場合：南側の1階を主たる居室、2階をその他の居室とする

![image.png](attachment:image.png)

In [12]:
# 戸建住宅の面積算出
# 南側1階主たる居室、2階その他の居室
def get_area_detached_main_equal_to_other ( area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
                                            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows ) :

    area={}
    
    # 窓面積
    # 北窓は非居室
    # 南窓は1階主たる居室、南側2階その他の居室
    # 東窓・西窓は北側非居室、南側1階主たる居室、2階その他の居室
    area['windows']={
        'n' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_windows['n']
        },
        's' : {
            'main_occupant_room'  : area_windows['s'] * ratio_floor_height[0] ,
            'other_occupant_room' : area_windows['s'] * ratio_floor_height[1] ,
            'non_occupant_room'   : 0
        },
        'e' : {
            'main_occupant_room'  : area_windows['e'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[0] ,
            'other_occupant_room' : area_windows['e'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[1] ,
            'non_occupant_room'   : area_windows['e'] * ratio_length_NS_n
        },
        'w' : {
            'main_occupant_room'  : area_windows['w'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[0] ,
            'other_occupant_room' : area_windows['w'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[1] ,
            'non_occupant_room'   : area_windows['w'] * ratio_length_NS_n
        },
        'top' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : area_windows['top'] * ( 1 - ratio_length_NS_n ),
            'non_occupant_room'   : area_windows['top'] * ratio_length_NS_n
        }
    }

    # 一般部位面積
    area['general_parts']={
        'ceiling' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : (length_narrow - length_NS_n) * length_long / ratio_floor_r - area['windows']['top']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * length_long / ratio_floor_r - area['windows']['top']['non_occupant_room']
        },
        'floor' : {
            'main_occupant_room'  : (length_narrow - length_NS_n) * length_long / ratio_floor_r,
            'other_occupant_room' : 0,
            'non_occupant_room'   : length_NS_n * length_long / ratio_floor_r
        },
        'wall_n': {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_NS - area['windows']['n']['non_occupant_room']
        },
        'wall_s': {
            'main_occupant_room'  : length_long * floor_height[0] - area['windows']['s']['main_occupant_room'],
            'other_occupant_room' : length_long * floor_height[1] - area['windows']['s']['other_occupant_room'],
            'non_occupant_room'   : 0
        },
        'wall_e': {
            'main_occupant_room'  : ( length_narrow - length_NS_n ) * floor_height[0] - area['windows']['e']['main_occupant_room'],
            'other_occupant_room' : ( length_narrow - length_NS_n ) * floor_height[1] - area['windows']['e']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * height - area['windows']['e']['non_occupant_room']
        },
        'wall_w': {
            'main_occupant_room'  : ( length_narrow - length_NS_n ) * floor_height[0] - area['windows']['w']['main_occupant_room'],
            'other_occupant_room' : ( length_narrow - length_NS_n ) * floor_height[1]- area['windows']['w']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * height - area['windows']['w']['non_occupant_room']                    
        }
    }

    # 内壁面積
    area['inner_walls']={
        'main to other'      : length_long * ( length_narrow - length_NS_n ) ,
        'main to lonliving'  : length_long * floor_height[0] ,
        'other to nonliving' : length_long * floor_height[1] 
    }

    return area

#### Function2

主たる居室の床面積がその他の居室の床面積より大きい場合：北側を非居室、2階の南西をその他の居室とする

![image.png](attachment:image.png)

In [13]:
# 戸建住宅の面積算出
# 2階南西その他の居室
def get_area_detached_main_larger_than_other ( area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
                                            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows ) :
    
    length_EW_o= area_floor_r['other_occupant_room'] / ( length_narrow - length_NS_n )
 
    area={}
      
    # 窓面積
    # 北窓は非居室
    # 南窓は2階南西その他の居室、他は主たる居室、
    # 東窓は北側非居室、南側主たる居室
    # 西窓は北側非居室、南側1階主たる居室、南側2階その他の居室
    area['windows']={
        'n' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_windows['n']
        },
        's' : {
            'main_occupant_room'  : area_windows['s'] * (1 - length_EW_o * floor_height[1]/ area_NS) ,
            'other_occupant_room' : area_windows['s'] * (length_EW_o * floor_height[1])/ area_NS ,
            'non_occupant_room'   : 0
        },
        'e' : {
            'main_occupant_room'  : area_windows['e'] * ( 1 - ratio_length_NS_n ),
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_windows['e'] * ratio_length_NS_n
        },
        'w' : {
            'main_occupant_room'  : area_windows['w'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[0] ,
            'other_occupant_room' : area_windows['w'] *  ( 1 - ratio_length_NS_n ) * ratio_floor_height[1] ,
            'non_occupant_room'   : area_windows['w'] * ratio_length_NS_n
        },
        'top' : {
            'main_occupant_room'  : area_windows['top'] * ( 1 - ratio_length_NS_n ) * ( 1 - length_EW_o/length_long ),
            'other_occupant_room' : area_windows['top'] * ( 1 - ratio_length_NS_n ) * ( length_EW_o/length_long ),
            'non_occupant_room'   : area_windows['top'] * ratio_length_NS_n
        }
    }

    # 外壁面積
    area['general_parts']={
        'ceiling' : {
            'main_occupant_room'  : (length_narrow - length_NS_n) * (length_long - length_EW_o) / ratio_floor_r - area['windows']['top']['main_occupant_room'],
            'other_occupant_room' : (length_narrow - length_NS_n) * length_EW_o / ratio_floor_r - area['windows']['top']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * length_long / ratio_floor_r - area['windows']['top']['non_occupant_room']
        },
        'floor' : {
            'main_occupant_room'  : (length_narrow - length_NS_n) * length_long / ratio_floor_r,
            'other_occupant_room' : 0,
            'non_occupant_room'   : length_NS_n * length_long / ratio_floor_r
        },
        'wall_n': {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_NS - area['windows']['n']['non_occupant_room']
        },
        'wall_s': {
            'main_occupant_room'  : (area_NS - length_EW_o * floor_height[1]) - area['windows']['s']['main_occupant_room'],
            'other_occupant_room' : length_EW_o * floor_height[1] - area['windows']['s']['other_occupant_room'],
            'non_occupant_room'   : 0
        },
        'wall_e': {
            'main_occupant_room'  : ( length_narrow - length_NS_n ) * height - area['windows']['e']['main_occupant_room'],
            'other_occupant_room' : 0,
            'non_occupant_room'   : length_NS_n * height - area['windows']['e']['non_occupant_room']
        },
        'wall_w': {
            'main_occupant_room'  : ( length_narrow - length_NS_n ) * floor_height[0] - area['windows']['w']['main_occupant_room'],
            'other_occupant_room' : ( length_narrow - length_NS_n ) * floor_height[1]- area['windows']['w']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * height - area['windows']['w']['non_occupant_room']                    
        }
    }

    # 内壁面積
    area['inner_walls']={
        'main to other'      : ( length_narrow - length_NS_n ) * floor_height[1] + ( length_narrow - length_NS_n ) * length_EW_o ,
        'main to lonliving'  : area_NS - length_EW_o * floor_height[1] ,
        'other to nonliving' : length_EW_o * floor_height[1] 
    }

    return area

#### Function3

主たる居室の床面積がその他の居室の床面積より小さい場合：北側を非居室、1階の南東を主たる居室とする

![image.png](attachment:image.png)

In [14]:
# 戸建住宅の面積算出
# 1階南東主たる居室
def get_area_detached_main_smaller_than_other ( area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
                                            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows ) :
    
    length_EW_m= area_floor_r['main_occupant_room'] / ( length_narrow - length_NS_n )
 
    area={}
    
    # 窓面積
    # 北窓は非居室
    # 南窓は1階南東主たる居室、他はその他の居室、
    # 東窓は北側非居室、南側1階主たる居室、南側2階その他の居室
    # 西窓は北側非居室、南側その他の居室
    area['windows']={
        'n' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_windows['n']
        },
        's' : {
            'main_occupant_room'  : area_windows['s'] * (length_EW_m * floor_height[0])/ area_NS ,
            'other_occupant_room' : area_windows['s'] * (1 - length_EW_m * floor_height[0]/ area_NS) ,
            'non_occupant_room'   : 0
        },
        'e' : {
            'main_occupant_room'  : area_windows['e'] * ( 1 - ratio_length_NS_n ) * ratio_floor_height[0] ,
            'other_occupant_room' : area_windows['e'] *  ( 1 - ratio_length_NS_n ) * ratio_floor_height[1] ,
            'non_occupant_room'   : area_windows['e'] * ratio_length_NS_n
        },
        'w' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : area_windows['w'] * ( 1 - ratio_length_NS_n ),
            'non_occupant_room'   : area_windows['w'] * ratio_length_NS_n
        },
        'top' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : area_windows['top'] * ( 1 - ratio_length_NS_n ),
            'non_occupant_room'   : area_windows['top'] * ratio_length_NS_n
        }
    }

    # 外壁面積
    area['general_parts']={
        'ceiling' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : (length_narrow - length_NS_n) * length_long / ratio_floor_r - area['windows']['top']['other_occupant_room'],
            'non_occupant_room'   :length_NS_n * length_long / ratio_floor_r - area['windows']['top']['non_occupant_room']
        },
        'floor' : {
            'main_occupant_room'  : (length_narrow - length_NS_n) * length_EW_m / ratio_floor_r,
            'other_occupant_room' : (length_narrow - length_NS_n) * (length_long - length_EW_m) / ratio_floor_r,
            'non_occupant_room'   : length_NS_n * length_long / ratio_floor_r
        },
        'wall_n': {
            'main_occupant_room'  : 0,
            'other_occupant_room' : 0,
            'non_occupant_room'   : area_NS - area['windows']['n']['non_occupant_room']
        },
        'wall_s': {
            'main_occupant_room'  : length_EW_m * floor_height[0] - area['windows']['s']['main_occupant_room'],
            'other_occupant_room' : (area_NS - length_EW_m * floor_height[0]) - area['windows']['s']['other_occupant_room'],
            'non_occupant_room'   : 0
        },
        'wall_e': {
            'main_occupant_room'  : ( length_narrow - length_NS_n ) * floor_height[0] - area['windows']['e']['main_occupant_room'],
            'other_occupant_room' : ( length_narrow - length_NS_n ) * floor_height[1]- area['windows']['e']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * height - area['windows']['e']['non_occupant_room']
        },
        'wall_w': {
            'main_occupant_room'  : 0,
            'other_occupant_room' : ( length_narrow - length_NS_n ) * height - area['windows']['w']['other_occupant_room'],
            'non_occupant_room'   : length_NS_n * height - area['windows']['w']['non_occupant_room']                    
        }
    }

    # 内壁面積
    area['inner_walls']={
        'main to other'      : ( length_narrow - length_NS_n ) * floor_height[0] + ( length_narrow - length_NS_n ) * length_EW_m ,
        'main to lonliving'  : length_EW_m * floor_height[0] ,
        'other to nonliving' : area_NS - length_EW_m * floor_height[0] 
    }

    return area

#### Function4

方位別開口部面積が各方位の外皮面積を超過する場合、南、東、北、西、天井の順に、各方位の外皮面積を上限として、開口部面積を割り付ける。

戸建住宅の各部位の面積を、以下の場合に応じて、算出する。  
　主たる居室とその他の居室の床面積が等しい場合：北側を非居室、南側の1階を主たる居室、2階をその他の居室とする  
　主たる居室の床面積がその他の居室の床面積より大きい場合：北側を非居室、2階の南西をその他の居室とする  
　主たる居室の床面積がその他の居室の床面積より小さい場合：北側を非居室、1階の南東を主たる居室とする  

In [15]:
# 戸建住宅の面積算出
def get_area_detached( area_floor_total, area_floor, area_floor_r, n, height, floor_height, length_long, length_narrow, area_windows) :
    
    # 延床面積に対する用途別床面積の割合
    ratio_floor = { x : area_floor[x] / area_floor_total for x in area_floor.keys()}
 
    # アスペクト比を上限値により調整している場合、調整前の床面積に対する調整後の床面積の割合
    ratio_floor_r = sum( area_floor_r[x] for x in area_floor_r ) / sum( area_floor[x] for x in area_floor )
    
    # 建物高に対する階高の割合
    ratio_floor_height = [ floor_height[0] / height, floor_height[1] / height ]
    
    # 南北面・東西面面積, ㎡
    area_NS = length_long * height
    area_EW = length_narrow * height
    
    # 非居室南北方向長さ, m
    length_NS_n = (area_floor_r['non_occupant_room'])/n/length_long
    ratio_length_NS_n = length_NS_n / length_narrow
    
    # 窓面積の調整 南、東、北、西、天井の順に割付
    area_windows = adjust_window_area( area_windows, 
                                      [{'direction':  's','area_max':area_EW},
                                       {'direction':  'e','area_max':area_NS},
                                       {'direction':  'n','area_max':area_EW},
                                       {'direction':  'w','area_max':area_NS},
                                       {'direction':'top','area_max':length_long * length_narrow / ratio_floor_r }] )
    
    # 面積算出
    # 南側1階主たる居室、2階その他の居室
    if area_floor['main_occupant_room'] == area_floor['other_occupant_room'] :
        area = get_area_detached_main_equal_to_other( \
            area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows )
    # 2階南西その他の居室
    elif area_floor['main_occupant_room'] > area_floor['other_occupant_room'] :
        area = get_area_detached_main_larger_than_other( \
            area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows )
    # 1階南東主たる居室
    elif area_floor['main_occupant_room'] < area_floor['other_occupant_room'] :
        area = get_area_detached_main_smaller_than_other( \
            area_floor_r, ratio_floor_r, height, floor_height, ratio_floor_height, \
            length_long, length_narrow, length_NS_n, ratio_length_NS_n, area_NS, area_windows )
        
    return area

### 8) 共同住宅の面積の算出

#### Outline

共同住宅の各部位の面積を算出する。  
方位別開口部面積が各方位の外皮面積を超過する場合、南、北、西、東、天井の順に、各方位の外皮面積を上限として、開口部面積を割り付ける。

![image.png](attachment:image.png)

#### Function

In [16]:
# 共同住宅の面積算出
def get_area_condominium( area_floor_total, area_floor, area_floor_r, height, length_long, length_narrow, area_windows) :
    
    # 延床面積に対する用途別床面積の割合
    ratio_floor = { x : area_floor[x] / area_floor_total for x in area_floor.keys()}
 
    # アスペクト比を上限値により調整している場合、調整前の床面積に対する調整後の床面積の割合
    ratio_floor_r = sum( area_floor_r[x] for x in area_floor_r ) / sum( area_floor[x] for x in area_floor )
    
    area_horizontal = length_long * length_narrow
    area_NS = length_long * height
    area_EW = length_narrow * height

    # 窓面積の調整 南、北、西、東、天井の順に割付
    area_windows = adjust_window_area( area_windows, 
                                      [{'direction':  's','area_max':area_EW},
                                       {'direction':  'n','area_max':area_EW},
                                       {'direction':  'w','area_max':area_NS},
                                       {'direction':  'e','area_max':area_NS},
                                       {'direction':'top','area_max':length_long * length_narrow / ratio_floor_r }] )
    area={}
    
    # 窓面積
    # 北窓はその他の居室、南窓は主たる居室、東窓・西窓は各用途に床面積案分で割り付け
    area['windows']={
        'n' : {
            'main_occupant_room'  : 0,
            'other_occupant_room' : area_windows['n'],
            'non_occupant_room'   : 0
        },
        's' : {
            'main_occupant_room'  : area_windows['s'],
            'other_occupant_room' : 0,
            'nonl_occupant_room'  : 0
        },
        'e' : {
            'main_occupant_room'  : area_windows['e'] * ratio_floor['main_occupant_room'],
            'other_occupant_room' : area_windows['e'] * ratio_floor['other_occupant_room'],
            'non_occupant_room'   : area_windows['e'] * ratio_floor['non_occupant_room']
        },
        'w' : {
            'main_occupant_room'  : area_windows['w'] * ratio_floor['main_occupant_room'],
            'other_occupant_room' : area_windows['w'] * ratio_floor['other_occupant_room'],
            'non_occupant_room'   : area_windows['w'] * ratio_floor['non_occupant_room']
        },
        'top' : {
            'main_occupant_room'  : area_windows['top'] * ratio_floor['main_occupant_room'],
            'other_occupant_room' : area_windows['top'] * ratio_floor['other_occupant_room'],
            'non_occupant_room'   : area_windows['top'] * ratio_floor['non_occupant_room']
        }
    }

    # 壁面積
    # 北壁はその他の居室、南壁は主たる居室、天井・床・東壁・西壁は各用途に床面積案分で割り付け
    area['general_parts']={
        'ceiling' : {
            'main_occupant_room'  : ( area_horizontal - area_windows['top'] ) * ratio_floor['main_occupant_room']  / ratio_floor_r,
            'other_occupant_room' : ( area_horizontal - area_windows['top'] ) * ratio_floor['other_occupant_room'] / ratio_floor_r,
            'nonl_occupant_room'  : ( area_horizontal - area_windows['top'] ) * ratio_floor['non_occupant_room']   / ratio_floor_r
        },
        'floor' : {
            'main_occupant_room'  : area_horizontal * ratio_floor['main_occupant_room']  / ratio_floor_r,
            'other_occupant_room' : area_horizontal * ratio_floor['other_occupant_room'] / ratio_floor_r,
            'non_occupant_room'   : area_horizontal * ratio_floor['non_occupant_room']   / ratio_floor_r
        },
        'wall_n': {
            'main_occupant_room'  : 0,
            'other_occupant_room' : area_EW - area_windows['n'],
            'non_occupant_room'   : 0
        },
        'wall_s': {
            'main_occupant_room'  : area_EW - area_windows['s'],
            'other_occupant_room' : 0,
            'nonl_occupant_room'  : 0
        },
        'wall_e': {
            'main_occupant_room' :     ( area_NS - area_windows['e'] ) * ratio_floor['main_occupant_room'],
            'other_occupant_room' :    ( area_NS - area_windows['e'] ) * ratio_floor['other_occupant_room'],
            'non_occupant_room' :( area_NS - area_windows['e'] ) * ratio_floor['non_occupant_room']
        },
        'wall_w': {
            'main_occupant_room' :     ( area_NS - area_windows['w'] ) * ratio_floor['main_occupant_room'],
            'other_occupant_room' :    ( area_NS - area_windows['w'] ) * ratio_floor['other_occupant_room'],
            'non_occupant_room' :( area_NS - area_windows['w'] ) * ratio_floor['non_occupant_room']
        }
    }
    
    # 内壁面積
    area['inner_walls']={
        'main to other' :      area_EW,
        'main to lonliving' :  area_EW,
        'other to nonliving' : area_EW
    }
        
    return area

### 9) 部位の面積を取得する関数

#### Outline

基準値計算用の住宅について、用途別床面積、幾何形状、開口部面積、各部位の面積を取得する関数を統合する。  
各部位の面積より外壁・開口部・内壁のリストを作成する。

#### Function

In [17]:
def get_area(d):
    
    # 用途別床面積の取得
    area_floor=get_area_floor(d['common'])

    # 幾何形状の取得
    n, height, floor_height, area_total_skin, length_narrow, length_long, area_floor_r = get_geometry( d, area_floor )
    
    # 開口部面積, ㎡
    area_windows = get_area_windows( d['common']['house_type'], area_total_skin )
    
    # 部位面積, ㎡
    if d['common']['house_type']== 'detached' :
        area = get_area_detached(d['common']['total_floor_area'], \
                                 area_floor, area_floor_r, n, height, floor_height, length_long, length_narrow, area_windows)
    elif d['common']['house_type']== 'condominium' :
        area = get_area_condominium(d['common']['total_floor_area'], \
                                    area_floor, area_floor_r, height, length_long, length_narrow, area_windows)
    
    # 外壁・開口部・内壁のリスト作成
    d = get_list(area)
   
    return d['general_parts'], d['windows'], d['inner_walls']

In [21]:
get_area({
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 60.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 120.0,
        'total_outer_skin_area': 360.0,
        'house_type': 'condominium' # 'detached','condominium'
    }
})

([{'name': 'ceiling_main_occupant_room',
   'general_part_type': 'ceiling',
   'next_space': 'outdoor',
   'external_surface_type': 'outdoor',
   'direction': 'top',
   'area': 60.0,
   'space_type': 'main_occupant_room'},
  {'name': 'ceiling_other_occupant_room',
   'general_part_type': 'ceiling',
   'next_space': 'outdoor',
   'external_surface_type': 'outdoor',
   'direction': 'top',
   'area': 30.0,
   'space_type': 'other_occupant_room'},
  {'name': 'ceiling_nonl_occupant_room',
   'general_part_type': 'ceiling',
   'next_space': 'outdoor',
   'external_surface_type': 'outdoor',
   'direction': 'top',
   'area': 30.0,
   'space_type': 'nonl_occupant_room'},
  {'name': 'floor_main_occupant_room',
   'general_part_type': 'floor',
   'next_space': 'outdoor',
   'external_surface_type': 'outdoor',
   'direction': 'bottom',
   'area': 60.0,
   'space_type': 'main_occupant_room'},
  {'name': 'floor_other_occupant_room',
   'general_part_type': 'floor',
   'next_space': 'outdoor',
   'ex